In [1]:
# PyTorch张量类型转换与维度变换学习笔记
# 涵盖:数据类型转换、张量与其他数据结构互转、形状变换、维度操作
import torch
import numpy as np

1. 张量元素类型转换

In [2]:
# (1) 使用.type()转换数据类型
# 功能:将张量转换为指定类型,返回新张量(不修改原张量)
# 示例:int64→float32, 用于模型输入数据类型统一
tensor1 = torch.tensor([1, 2, 3])  # 默认int64
print("原始类型:", tensor1.dtype)
tensor2 = tensor1.type(torch.float32)  # 转换为float32
print("转换后类型:", tensor2.dtype)
print("转换后值:", tensor2)  # tensor([1., 2., 3.])

原始类型: torch.int64
转换后类型: torch.float32
转换后值: tensor([1., 2., 3.])


In [3]:
# (2) 使用快捷方法转换类型
# .int()→int32, .long()→int64, .float()→float32, .double()→float64, .half()→float16
# 应用:标签索引用.long(), 权重用.float(), 混合精度训练用.half()
tensor1 = torch.tensor([1.5, 2.7, 3.9])
print("原始float:", tensor1)
print("转int32:", tensor1.int())      # tensor([1, 2, 3])
print("转int64:", tensor1.long())     # tensor([1, 2, 3])
print("转float16:", tensor1.half())   # tensor([1.5000, 2.7000, 3.9000], dtype=float16)

原始float: tensor([1.5000, 2.7000, 3.9000])
转int32: tensor([1, 2, 3], dtype=torch.int32)
转int64: tensor([1, 2, 3])
转float16: tensor([1.5000, 2.6992, 3.9004], dtype=torch.float16)


In [4]:
# (3) 使用.to()转换类型和设备(推荐方法)
# 优势:同时支持类型转换和设备迁移(CPU↔GPU), 代码更统一
# 示例:.to(torch.float32)转类型, .to('cuda')转GPU, .to('cuda', dtype=torch.float16)同时转换
tensor1 = torch.tensor([1, 2, 3])
tensor2 = tensor1.to(torch.float32)  # 转换为float32
print("使用.to()转换:", tensor2, tensor2.dtype)
# tensor3 = tensor1.to('cuda')  # 转移到GPU(需要GPU环境)
# 转化为复数
tensor1 = torch.tensor([1, 2, 3])
tensor2 = tensor1.to(torch.complex64)
print("转为复数:", tensor2)

使用.to()转换: tensor([1., 2., 3.]) torch.float32
转为复数: tensor([1.+0.j, 2.+0.j, 3.+0.j])


2. 张量与其他数据结构转换

In [5]:
# (1) 张量 ↔ NumPy数组
# torch.from_numpy(ndarray):共享内存,修改一个会影响另一个(仅CPU张量)
# tensor.numpy():转为NumPy数组,也共享内存
# 应用:与NumPy生态交互,如使用matplotlib绘图、sklearn预处理

# NumPy → Tensor (共享内存)
np_array = np.array([1, 2, 3])
tensor1 = torch.from_numpy(np_array)
print("从NumPy创建:", tensor1)
np_array[0] = 999  # 修改NumPy数组
print("修改NumPy后tensor:", tensor1)  # tensor也变了!tensor([999, 2, 3])

# Tensor → NumPy (共享内存)
tensor2 = torch.tensor([4, 5, 6])
np_array2 = tensor2.numpy()
print("转为NumPy:", np_array2)
tensor2[0] = 888  # 修改张量
print("修改tensor后NumPy:", np_array2)  # NumPy也变了!array([888, 5, 6])

从NumPy创建: tensor([1, 2, 3])
修改NumPy后tensor: tensor([999,   2,   3])
转为NumPy: [4 5 6]
修改tensor后NumPy: [888   5   6]


In [6]:
# (2) 张量 ↔ Python列表
# tensor.tolist():转为Python原生列表,不共享内存
# torch.tensor(list):从列表创建张量,复制数据
# 应用:数据持久化(JSON存储)、小规模数据调试查看

# Tensor → List
tensor1 = torch.tensor([[1, 2], [3, 4]])
py_list = tensor1.tolist()
print("转为列表:", py_list)  # [[1, 2], [3, 4]]
print("列表类型:", type(py_list))  # <class 'list'>

# List → Tensor
tensor2 = torch.tensor(py_list)
print("从列表创建:", tensor2)

转为列表: [[1, 2], [3, 4]]
列表类型: <class 'list'>
从列表创建: tensor([[1, 2],
        [3, 4]])


In [7]:
# (3) 获取张量的标量值
# .item():仅适用于单元素张量,返回Python标量
# 应用:提取损失值、准确率等单个数值用于日志记录
# 注意:多元素张量调用.item()会报错
tensor1 = torch.tensor(3.14)  # 0维标量张量
value = tensor1.item()
print("标量值:", value)  # 3.14
print("值类型:", type(value))  # <class 'float'>

# 实际应用示例
loss = torch.tensor(0.523)  # 假设这是损失值
print(f"Epoch Loss: {loss.item():.4f}")  # 格式化输出

标量值: 3.140000104904175
值类型: <class 'float'>
Epoch Loss: 0.5230


3. 张量形状变换

In [8]:
# (1) .view()重塑张量形状
# 功能:返回新视图(共享数据),不复制内存,要求张量在内存中连续
# 参数:-1表示自动推断该维度大小
# 应用:全连接层输入展平(batch,C,H,W)→(batch,C*H*W)
# 示例:6个元素可重塑为(2,3)、(3,2)、(6,)、(1,6)等
tensor1 = torch.arange(12)  # tensor([0,1,2,...,11])
print("原始形状:", tensor1.shape)  # torch.Size([12])

tensor2 = tensor1.view(3, 4)  # 重塑为3x4
print("重塑为3x4:\n", tensor2)

tensor3 = tensor1.view(2, -1)  # 2行,-1自动计算为6列
print("重塑为2x?:\n", tensor3, "形状:", tensor3.shape)

原始形状: torch.Size([12])
重塑为3x4:
 tensor([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11]])
重塑为2x?:
 tensor([[ 0,  1,  2,  3,  4,  5],
        [ 6,  7,  8,  9, 10, 11]]) 形状: torch.Size([2, 6])


In [9]:
# (2) .reshape()重塑张量(推荐)
# 区别view:reshape在张量不连续时会自动复制,更安全但可能慢
# 应用:与view相同,但适用于转置等操作后的张量
# 选择建议:能用view就用view(快),不确定连续性用reshape(安全)
tensor1 = torch.arange(12)
tensor2 = tensor1.reshape(3, 4)
print("reshape结果:\n", tensor2)

# reshape可处理转置后的张量
tensor3 = tensor2.t()  # 转置,在内存中不连续
tensor4 = tensor3.reshape(2, 6)  # reshape成功
print("转置后reshape:\n", tensor4)

reshape结果:
 tensor([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11]])
转置后reshape:
 tensor([[ 0,  4,  8,  1,  5,  9],
        [ 2,  6, 10,  3,  7, 11]])


In [10]:
# (3) .flatten()展平张量
# 功能:将多维张量展平为一维,默认从第0维开始
# 参数:start_dim, end_dim指定展平的维度范围
# 应用:CNN输出→全连接层输入(保留batch维度)
# 示例:(batch=2, C=3, H=4, W=4)→flatten(1)→(2, 48)
tensor1 = torch.randn(2, 3, 4)  # 形状(2,3,4)
print("原始形状:", tensor1.shape)

tensor2 = tensor1.flatten()  # 全部展平
print("全部展平:", tensor2.shape)  # torch.Size([24])

tensor3 = tensor1.flatten(start_dim=1)  # 保留第0维(batch),展平后续维度
print("保留batch展平:", tensor3.shape)  # torch.Size([2, 12])

原始形状: torch.Size([2, 3, 4])
全部展平: torch.Size([24])
保留batch展平: torch.Size([2, 12])
